<a href="https://colab.research.google.com/github/dwhite25/MSG-CcGAN/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
main executable for GAN. 

eventually this will be replaced with another program that pulls and stores data 
from .h5 files as time series data, then runs the GAN on it. this version is simply
for using with images, to test the main implementation of MSG-GAN.

Just about everything should be included inside the GAN class, so here we really
just need to define some main vars, do some imports, and call the class. 
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/GWPAC/MSG-cGAN/Ours/classes')
!pip install tensorflow-addons

In [ ]:
import tensorflow as tf 
import dataset as ds
import msggan as gan
from pathlib import Path
import csv

In [ ]:
# define some variables/argumens to be used elsewhere 
# (some of these might be put inside the class rather than here)

def main():
    loadweights = True
    usebias     = False
    nrns        = 64
    glr         = 1e-2
    dlr         = 1e-2
    r1g         = 1e-1
    epsln       = 1e-3
    res         = 8192
    bsize       = 32
    ksize       = 9

    pathway = 'drive/MyDrive/GWPAC/MSG-cGAN/Ours/ins/time_series/has_postmergers/sims'
    metadat = 'drive/MyDrive/GWPAC/MSG-cGAN/Ours/ins/time_series/has_postmergers/METADATA.csv'
    outpath = f'drive/MyDrive/GWPAC/MSG-cGAN/Ours/outs/tseq_{res}d_{nrns}n_{glr}_{dlr}_{r1g}r_{epsln}e_{ksize}k_c'
    weights = f'drive/MyDrive/GWPAC/MSG-cGAN/Ours/outs/tseq_{res}d_{nrns}n_{glr}_{dlr}_{r1g}r_{epsln}e_{ksize}k_b/weights'
    
    Path(outpath).mkdir(exist_ok=True)
    with open(outpath + '/attrs.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['loadweights', loadweights])
        writer.writerow(['neurons', nrns])
        writer.writerow(['g learning rate', glr])
        writer.writerow(['d learning rate', dlr])
        writer.writerow(['r1gamma', r1g])
        writer.writerow(['epsilon', epsln])
        writer.writerow(['resolution', res])
        writer.writerow(['batch size', bsize])

    # load the dataset
    dataset = ds.Dataset(batch_size=bsize, pathway=pathway, mdpath=metadat, outpath=outpath, nlabels=4, endres=res)
    # call the GAN class
    # msgg = gan.MSG_GAN_ts(neurs=nrns, endres=dataset.endres, g_lr=glr, d_lr=dlr, r1_gamma=r1g, epsilon=epsln, 
    #                       outpath=outpath, epochs=500000, nchannels=2, ksize=ksize, usebias=usebias)
    msgg = gan.MSG_CcGAN_ts(neurs=nrns, endres=dataset.endres, g_lr=glr, d_lr=dlr, r1_gamma=r1g, epsilon=epsln, 
                            outpath=outpath, epochs=500000, nchannels=2, ksize=ksize, usebias=usebias)
    # continue from previous save point?
    if loadweights == True:
        msgg.D.load_weights(weights + '/discriminator41000.h5')
        msgg.G.load_weights(weights + '/generator41000.h5')
    # start the training loop
    msgg.train(dataset)

    # make sample waveforms 
    # msgg.makewaves(dataset)


if __name__ == "__main__":
    main()